# collection with the results of step 1:


In [16]:
from lsst.daf.butler import Butler
import pandas as pd
import re

In [2]:
butler = Butler('/repo/main', collections=['u/sierrav/PDR2/w_2024_03_fitAffineWCS/DM-42607'])
#butler = Butler('/repo/main', collections=['/u/sierrav/cm/pandatesting/DM-42607_test1/inputs'])
#butler = Butler('/repo/main', collections=['HSC/runs/PDR2/v24_1_0/DM-39132'])

### example table from PDR2 reprocessing:
```
collection='HSC/raw/PDR2/v24_1_0/DM-39132/input'
1783445 = 17315 visits * 103 detectors
                        SUCCEEDED FAILED EXPECTED
----------------------- --------- ------ --------
TOTAL                     1778310   5135  1783445
----------------------- --------- ------ --------
isr                       1779973   3472  1783445
characterizeImage         1778481   1492  1779973
calibrate                 1778310    171  1778481 
writePreSourceTable       1778310      0  1778310
transformPreSourceTable   1778310      0  1778310
```

In [11]:
#resFirstFalse = butler.registry.queryDatasets('postISRCCD', dimensions=['visit', 'detector'], findFirst=False)
#res.count(exact=False)

In [12]:
#resFirstFalseSet = set([ r for r in resFirstFalse])
#len(resFirstFalseSet)

1485718

In [3]:
results = butler.registry.queryDataIds(datasets='raw', dimensions=['visit', 'detector'], 
                                       where="instrument = 'HSC' and detector != 9 and detector < 140 and band in ('g', 'r', 'i', 'z', 'y')")
rawSet = set([r for r in results])
len(rawSet)

2064275

In [4]:
results = butler.registry.queryDataIds(datasets='postISRCCD', dimensions=['visit', 'detector'])
resSet = set([ r for r in results])
len(resSet)

599882

In [5]:
results = butler.registry.queryDataIds(datasets='icSrc', dimensions=['visit','detector'])
icSrcSet = set([ r for r in results])
len(icSrcSet)

550362

In [6]:
results = butler.registry.queryDataIds(datasets='calexp', dimensions=['visit','detector'])
calexpSet = set([ r for r in results])
len(calexpSet)

542272

In [7]:
results = butler.registry.queryDataIds(datasets='preSourceTable', dimensions=['visit','detector'])
stSet = set([ r for r in results])
len(stSet)

542272

In [8]:
failedIsrSet = (rawSet-resSet)
failedCharSet = (resSet-icSrcSet)
failedCalibrateSet = (icSrcSet - stSet)

In [ ]:
errors = []
noLogs = []
noError = []
for i, dataId in enumerate(failedIsrSet):
    try:
        log = butler.get('isr_log', dataId={'exposure': dataId['visit'], 'detector':  dataId['detector']})
    except:
        noLogs.append(dataId)
        #print(i, "noLog")
        continue
    appendedError = False
    for l in log:
        if l.levelname == "ERROR":
            errors.append(l)
            #print(i, "appending Error")
            appendedError = True
    if not appendedError:
        noError.append(l)
        #print(i, "noError")
    if i > 100: # if you just want to check it out a few without reading in a 50000 logs
        break

In [ ]:
errors = []
noLogs = []
noError = []
for i, dataId in enumerate(failedCharSet):
    try:
        log = butler.get('characterizeImage_log', dataId={'visit': dataId['visit'], 'detector':  dataId['detector']})
    except:
        noLogs.append(dataId)
        #print(i, "noLog")
        continue
    appendedError = False
    for l in log:
        if l.levelname == "ERROR":
            errors.append(l)
            #print(i, "appending Error")
            appendedError = True
    if not appendedError:
        noError.append(l)
        #print(i, "noError")
    if i > 100: # if you just want to check it out a few without reading in a 50000 logs
        break

In [9]:
errors = []
noLogs = []
noError = []

total = len(failedCalibrateSet)
for i, dataId in enumerate(failedCalibrateSet):
    try:
        log = butler.get('calibrate_log', dataId={'visit': dataId['visit'], 'detector':  dataId['detector']})
    except:
        noLogs.append(dataId)
        # print(i, "noLog")
        continue
    appendedError = False
    for l in log:
        if l.levelname == "ERROR":
            errors.append(l)
            # print(i, "appending Error")
            appendedError = True
    if not appendedError:
        noError.append(l)
        # print(i, "noError")
    if i % 1000 == 0:
        print(f'{i} of {total}')

In [14]:
for l in log:
    print(l)

INFO 2024-02-06T23:11:53.889086+00:00 lsst.ctrl.mpexec.singleQuantumExecutor singleQuantumExecutor.py:204 - Preparing execution of quantum for label=calibrate dataId={instrument: 'HSC', detector: 109, visit: 944, band: 'i', physical_filter: 'HSC-I'}.
VERBOSE 2024-02-06T23:11:53.893072+00:00 lsst.daf.butler.datastores.fileDatastore fileDatastore.py:1472 - Number of datasets found in datastore: 2 out of 2 datasets checked.
INFO 2024-02-06T23:11:53.899630+00:00 lsst.ctrl.mpexec.singleQuantumExecutor singleQuantumExecutor.py:256 - Constructing task and executing quantum for label=calibrate dataId={instrument: 'HSC', detector: 109, visit: 944, band: 'i', physical_filter: 'HSC-I'}.
INFO 2024-02-06T23:11:55.030985+00:00 lsst.calibrate.detection detection.py:563 - Setting factor for negative detections equal to that for positive detections: 1.000000
INFO 2024-02-06T23:11:55.490821+00:00 lsst.calibrate.detection detection.py:653 - Detected 22075 positive peaks in 183 footprints and 242 negative

In [17]:
# remove all digits
messages = [re.sub('\d', '', e.message) for e in errors]
df = pd.DataFrame(messages)
df.columns = ["message"]

mdc = [e.MDC for e in errors]
df['mdc'] = mdc
df2 = df.groupby('message')['mdc'].apply(list).reset_index()

In [21]:
df.groupby('message').count()

,mdc
message,
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'g', physical_filter: 'HSC-G'} failed. Exception IndexError: arrays used as indices must be of integer (or boolean) type",1
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'g', physical_filter: 'HSC-G'} failed. Exception RuntimeError: No matches to use for photocal",33
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'g', physical_filter: 'HSC-G'} failed. Exception RuntimeError: WCS fit failed for {instrument: 'HSC', detector: , visit: , band: 'g', physical_filter: 'HSC-G'} and requireAstrometry is True.",1202
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'i', physical_filter: 'HSC-I'} failed. Exception IndexError: arrays used as indices must be of integer (or boolean) type",25
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'i', physical_filter: 'HSC-I'} failed. Exception RuntimeError: No matches to use for photocal",262
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'i', physical_filter: 'HSC-I'} failed. Exception RuntimeError: WCS fit failed for {instrument: 'HSC', detector: , visit: , band: 'i', physical_filter: 'HSC-I'} and requireAstrometry is True.",1187
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'r', physical_filter: 'HSC-R'} failed. Exception IndexError: arrays used as indices must be of integer (or boolean) type",6
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'r', physical_filter: 'HSC-R'} failed. Exception RuntimeError: No matches to use for photocal",49
"Execution of task 'calibrate' on quantum {instrument: 'HSC', detector: , visit: , band: 'r', physical_filter: 'HSC-R'} failed. Exception RuntimeError: WCS fit failed for {instrument: 'HSC', detector: , visit: , band: 'r', physical_filter: 'HSC-R'} and requireAstrometry is True.",2032


In [ ]:
# How to reproduce:
# pipetask run -b /sdf/group/rubin/repo/main -i HSC/raw/all,HSC/runs/PDR2/v24_1_0/DM-39132/ancillary -o u/yusra/HSC/PDR2/how_to_reproduce -d "exposure=84526 and detector=90" -p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#isr

In [23]:
# To make lists of dataIds to attach to tickets:

# first error 
INDEX = 1

with open("/sdf/group/rubin/u/yusra/dataIds.txt", "w") as f:
for element in  [e['LABEL'] for e in df2.iloc[INDEX].mdc]:
    # print(element.replace("calibrate:","").replace("...",""))
    f.write(element.replace("calibrate:","").replace("...",""))
    f.write("\n")

{instrument: 'HSC', detector: 106, visit: 34244, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 111, visit: 15302, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 110, visit: 42346, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 110, visit: 11702, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 109, visit: 85720, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 108, visit: 34238, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 110, visit: 11642, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 111, visit: 6336, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 106, visit: 25858, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 110, visit: 11684, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 110, visit: 9834, band: 'g', physical_filter: 'HSC-G'}
{instrument: 'HSC', detector: 106, visit: 85540, band: '